#### Pakkar

In [66]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

#### Innlestur gagna

In [45]:
train_data = pd.read_csv('data/train.csv',delimiter=',')

test_data = pd.read_csv('data/test.csv',delimiter=',')


### Gagnasettið
Hér má sjá gagnasettið sem við þjálfum módelið á. Við erum með comment_text og hann er búið að greina (0 eða 1) hvort hann sé
    - toxic
    - severe_toxic
    - obscene
    - threat
    - insult
    - identity_hate

In [48]:
display(train_data.head())

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [51]:
# til að vísa í fylkið dataset.values[index]

print('=================')
print('|','train_data shape:', train_data.shape)
print('|',train_data.shape[0], 'train samples')
print('|',test_data.shape[0], 'test samples')
print('=================')

| train_data shape: (159571, 8)
| 159571 train samples
| 153164 test samples


In [53]:
print(train_data['comment_text'][0:10])

0    Explanation\nWhy the edits made under my usern...
1    D'aww! He matches this background colour I'm s...
2    Hey man, I'm really not trying to edit war. It...
3    "\nMore\nI can't make any real suggestions on ...
4    You, sir, are my hero. Any chance you remember...
5    "\n\nCongratulations from me as well, use the ...
6         COCKSUCKER BEFORE YOU PISS AROUND ON MY WORK
7    Your vandalism to the Matt Shirvington article...
8    Sorry if the word 'nonsense' was offensive to ...
9    alignment on this subject and which are contra...
Name: comment_text, dtype: object


### Text preprocessing


In [54]:
def textpreprocess(sentence):
    sentence = sentence.lower()
    
    return sentence

In [70]:
for i in range(1):
    print(i,textpreprocess(train_data['comment_text'][i]),'\n')
    
tst = CountVectorizer(train_data['comment_text'][0],stop_words='english')
print(tst)

0 explanation
why the edits made under my username hardcore metallica fan were reverted? they weren't vandalisms, just closure on some gas after i voted at new york dolls fac. and please don't remove the template from the talk page since i'm retired now.89.205.38.27 

CountVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.int64'>, encoding='utf-8',
                input='Explanation\n'
                      'Why the edits made under my username Hardcore Metallica '
                      "Fan were reverted? They weren't vandalisms, just "
                      'closure on some GAs after I voted at New York Dolls '
                      "FAC. And please don't remove the template from the talk "
                      "page since I'm retired now.89.205.38.27",
                lowercase=True, max_df=1.0, max_features=None, min_df=1,
                ngram_range=(1, 1), preprocessor=None, stop_words='english',
                strip_accents=No